In [ ]:
from google.colab import files
files.upload()

In [2]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [3]:
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [4]:
!kaggle competitions download -c pubg-finish-placement-prediction

 65% 11.0M/17.0M [00:00<00:00, 16.8MB/s]
100% 17.0M/17.0M [00:00<00:00, 31.4MB/s]
 96% 233M/244M [00:05<00:00, 35.9MB/s]
100% 244M/244M [00:05<00:00, 44.6MB/s]
 89% 89.0M/100M [00:01<00:00, 35.8MB/s]
100% 100M/100M [00:01<00:00, 56.1MB/s] 


In [5]:
!unzip train_V2.csv.zip
!unzip test_V2.csv.zip

Archive:  train_V2.csv.zip
  inflating: train_V2.csv            
Archive:  test_V2.csv.zip
  inflating: test_V2.csv             


In [6]:
!ls

kaggle.json  sample_submission_V2.csv.zip  test_V2.csv.zip  train_V2.csv.zip
sample_data  test_V2.csv		   train_V2.csv


#Importing the dataset

Import the libraries

In [70]:
import numpy as np
import pandas as pd
import seaborn as sns

In [71]:
train=pd.read_csv("train_V2.csv")
train

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,1241,0,0,0.00,1306,squad-fpp,28,26,-1,0,0.0000,0,0.000,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,0,0,0,0.00,1777,squad-fpp,26,25,1484,0,0.0045,0,11.040,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,0,0,0,0.00,1318,duo,50,47,1491,0,0.0000,0,0.000,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,0,0,0,0.00,1436,squad-fpp,31,30,1408,0,0.0000,0,0.000,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,0,1,1,58.53,1424,solo-fpp,97,95,1560,0,0.0000,0,0.000,0,0,49.75,2,0,0.1875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4446961,afff7f652dbc10,d238e426f50de7,18492834ce5635,0,0,0.00,0,0,0,74,1029,0,0,0.00,1873,squad-fpp,29,28,-1,0,1292.0000,0,0.000,0,0,1019.00,3,1507,0.1786
4446962,f4197cf374e6c0,408cdb5c46b2ac,ee854b837376d9,0,1,44.15,0,0,0,69,0,0,0,0.00,1435,solo,93,93,1501,0,0.0000,0,0.000,0,0,81.70,6,0,0.2935
4446963,e1948b1295c88a,e26ac84bdf7cef,6d0cd12784f1ab,0,0,59.06,0,0,0,66,0,0,0,0.00,1321,squad-fpp,28,28,1500,0,0.0000,0,2.184,0,0,788.70,4,0,0.4815
4446964,cc032cdd73b7ac,c2223f35411394,c9c701d0ad758a,0,4,180.40,1,1,2,11,0,2,1,98.50,1373,squad-fpp,26,25,1418,2,0.0000,0,0.000,0,0,2748.00,8,0,0.8000


In [72]:
test=pd.read_csv("test_V2.csv")
test

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints
0,9329eb41e215eb,676b23c24e70d6,45b576ab7daa7f,0,0,51.46,0,0,0,73,0,0,0,0.00,1884,squad-fpp,28,28,1500,0,0.0,0,0.000,0,0,588.00,1,0
1,639bd0dcd7bda8,430933124148dd,42a9a0b906c928,0,4,179.10,0,0,2,11,0,2,1,361.90,1811,duo-fpp,48,47,1503,2,4669.0,0,0.000,0,0,2017.00,6,0
2,63d5c8ef8dfe91,0b45f5db20ba99,87e7e4477a048e,1,0,23.40,0,0,4,49,0,0,0,0.00,1793,squad-fpp,28,27,1565,0,0.0,0,0.000,0,0,787.80,4,0
3,cf5b81422591d1,b7497dbdc77f4a,1b9a94f1af67f1,0,0,65.52,0,0,0,54,0,0,0,0.00,1834,duo-fpp,45,44,1465,0,0.0,0,0.000,0,0,1812.00,3,0
4,ee6a295187ba21,6604ce20a1d230,40754a93016066,0,4,330.20,1,2,1,7,0,3,1,60.06,1326,squad-fpp,28,27,1480,1,0.0,0,0.000,0,0,2963.00,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1934169,a316c3a13887d5,80e82180c90951,f276da5fd73536,1,2,381.00,3,0,7,4,1294,4,2,69.27,1292,squad-fpp,22,20,-1,1,0.0,0,0.000,0,0,1799.00,5,1526
1934170,5312146b27d875,147264a127c56e,fb77cbfe094a6b,0,0,0.00,0,0,0,64,1449,0,0,0.00,1301,squad-fpp,27,26,-1,0,0.0,0,0.000,0,0,1195.00,3,1496
1934171,fc8818b5b32ad3,7bbb4779ebedd5,efb74d5f6e9779,0,0,91.96,0,0,3,24,0,1,1,25.38,1314,squad-fpp,29,29,1482,0,0.0,0,7.798,1,0,3327.00,3,0
1934172,a0f91e35f8458f,149a506a64c2b1,3a2cf335bc302f,1,2,138.60,0,0,12,25,0,1,1,67.89,1333,duo-fpp,49,48,1456,1,0.0,0,0.000,0,0,1893.00,4,0


In [73]:
train=train.sample(n=100000)

Deleting missing data

In [74]:
delete=train[train["winPlacePerc"].isnull()].index
delete

Int64Index([], dtype='int64')

In [75]:
train.drop(delete,inplace=True)

Removing the unnecessary columns from the dataset

In [76]:
y=train["winPlacePerc"]
train.drop(columns=["Id","groupId","matchId","winPlacePerc"],inplace=True)
test.drop(columns=["Id","groupId","matchId"],inplace=True)

 # Dividing into training set and test set

Dividing into training set and testing set (not normalised)

In [77]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train,y,test_size=0.2,random_state=1)

# Making pipelines

Pipelines are an excellent way to organise your code .So here we will make multiple pipelines with some differences and evaluate their performance

### **Pipeline 1:** <br>
> This pipeline uses **multiple linear regression** as the machine learning model to predict the winPlacePerc .



In [78]:
cat_col=[cname for cname in X_train.columns if X_train[cname].dtype=="object"]
num_col=[cname for cname in X_train.columns if X_train[cname].dtype !="object"]

Importing libraries

In [79]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

Preprocessing the data

In [80]:
num_tranformer=SimpleImputer(strategy="constant") #preprocessing numerical data
cat_tranformer=Pipeline(
    steps=[
           ("imputer",SimpleImputer(strategy="most_frequent")),
           ("onehot",OneHotEncoder(handle_unknown="ignore"))
    ]
) #preprocessing categorical data

In [81]:
preprocessor=ColumnTransformer(
    transformers=[
                 ("num",num_tranformer,num_col),
                 ("cat",cat_tranformer,cat_col)
    ]
)

Building the model

In [82]:
model=LinearRegression()

Making the pipeline

In [83]:
clf=Pipeline(steps=[
                    ('preprocessor',preprocessor),
                    ('model',model)
])

Fitting the model

In [84]:
clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='constant',
                                                                verbose=0),
                                                  ['assists', 'boosts',
                                                   'damageDealt', 'DBNOs',
                                                   'headshotKills', 'he

Making predictions

In [85]:
preds=clf.predict(X_test)

In [86]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test,preds)

0.08974781255004279

### **Pipeline 2** <br>
>Clearly,linear models are not working well for this problem. So this pipeline uses **Random Forest Regression** to make predictions.From the observation above we can use any data (normalised or non-normalised).

In [87]:
cat_col=[cname for cname in X_train.columns if X_train[cname].dtype=="object"]
num_col=[cname for cname in X_train.columns if X_train[cname].dtype !="object"]

In [88]:
num_col

['assists',
 'boosts',
 'damageDealt',
 'DBNOs',
 'headshotKills',
 'heals',
 'killPlace',
 'killPoints',
 'kills',
 'killStreaks',
 'longestKill',
 'matchDuration',
 'maxPlace',
 'numGroups',
 'rankPoints',
 'revives',
 'rideDistance',
 'roadKills',
 'swimDistance',
 'teamKills',
 'vehicleDestroys',
 'walkDistance',
 'weaponsAcquired',
 'winPoints']

In [89]:
cat_col

['matchType']

Importing libraries

In [90]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.ensemble import RandomForestRegressor

Preprocessing the data

In [91]:
num_tranformer=SimpleImputer(strategy="constant") #preprocessing numerical data
cat_tranformer=Pipeline(
    steps=[
           ("imputer",SimpleImputer(strategy="most_frequent")),
           ("onehot",OneHotEncoder(handle_unknown="ignore"))
    ]
) #preprocessing categorical data

In [92]:
preprocessor=ColumnTransformer(
    transformers=[
                 ("num",num_tranformer,num_col),
                 ("cat",cat_tranformer,cat_col)
    ]
)

Building the model

In [93]:
regressor=RandomForestRegressor(n_estimators=500,random_state=1,n_jobs=-1) # Random Forest Regressor

Making the pipeline

In [94]:
clf=Pipeline(steps=[
                    ('preprocessor',preprocessor),
                    ('model',regressor)
])

Fitting the model

In [95]:
clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='constant',
                                                                verbose=0),
                                                  ['assists', 'boosts',
                                                   'damageDealt', 'DBNOs',
                                                   'headshotKills', 'he

Making predictions

In [96]:
preds=clf.predict(X_test)

In [97]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test,preds)

0.0618662408685793

### **Pipeline 3**
This pipelines experiments with **esembling** . We can ensemble different models together and get predictions from them.

In [174]:
cat_col=[cname for cname in X_train.columns if X_train[cname].dtype=="object"]
num_col=[cname for cname in X_train.columns if X_train[cname].dtype !="object"]

In [175]:
num_col

['assists',
 'boosts',
 'damageDealt',
 'DBNOs',
 'headshotKills',
 'heals',
 'killPlace',
 'killPoints',
 'kills',
 'killStreaks',
 'longestKill',
 'matchDuration',
 'maxPlace',
 'numGroups',
 'rankPoints',
 'revives',
 'rideDistance',
 'roadKills',
 'swimDistance',
 'teamKills',
 'vehicleDestroys',
 'walkDistance',
 'weaponsAcquired',
 'winPoints']

In [176]:
cat_col

['matchType']

Importing libraries

In [177]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,VotingRegressor
from xgboost import XGBRegressor

Preprocessing the data

In [178]:
num_tranformer=SimpleImputer(strategy="constant") #preprocessing numerical data
cat_tranformer=Pipeline(
    steps=[
           ("imputer",SimpleImputer(strategy="most_frequent")),
           ("onehot",OneHotEncoder(handle_unknown="ignore"))
    ]
) #preprocessing categorical data

In [179]:
preprocessor=ColumnTransformer(
    transformers=[
                 ("num",num_tranformer,num_col),
                 ("cat",cat_tranformer,cat_col)
    ]
)

Building the model

In [180]:
r1=RandomForestRegressor(n_estimators=300,random_state=1,n_jobs=-1) # Random Forest Regressor
r2=XGBRegressor(n_estimators=300,learning_rate=0.05,objective='reg:squarederror')
r3=GradientBoostingRegressor(random_state=1,n_estimators=300,learning_rate=0.05)
regressor=VotingRegressor(
    estimators=[
     ("r1",r1),
     ("r2",r2),
     ("r3",r3)
    ]
)

Making the pipeline

In [181]:
clf=Pipeline(steps=[
                    ('preprocessor',preprocessor),
                    ('model',regressor)
])

Fitting the model

In [182]:
clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='constant',
                                                                verbose=0),
                                                  ['assists', 'boosts',
                                                   'damageDealt', 'DBNOs',
                                                   'headshotKills', 'he

Making predictions

In [183]:
preds=clf.predict(X_test)

In [184]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test,preds)

0.06361929520044292

# Evaluating performance

To evaluate the model we will look at various regression metrics and see what information they provide us with 

**Explained Variance**

In [185]:
from sklearn.metrics import explained_variance_score
explained_variance_score(y_test,preds)

0.915917620568069

From the above score we can see that our model explains 91% of the variance in the data.

**R2 score**

In [186]:
from sklearn.metrics import r2_score
r2_score(y_test,preds)

0.9159174619971939

R2 square is pretty close to the explained variance  score of the model.This model is pretty accurate but can be improved. 

**RMS (Root mean squared error)**

In [187]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test,preds))

0.08923642566854123

# Output on test data

Using the model to make the final predictions 

In [188]:
final_pred=clf.predict(test)

In [189]:
output=pd.DataFrame()
output["prediction"]=final_pred

In [190]:
output

,prediction
0,0.245904
1,0.888984
2,0.610949
3,0.549823
4,0.915475
...,...
1934169,0.707640
1934170,0.406476
1934171,0.870378
1934172,0.828065
